In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, interactive
from sklearn.preprocessing import PolynomialFeatures, Normalizer, MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, GridSearchCV
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn.apionly as sns
sns.set_style("whitegrid")
sns.set_context("poster")

# Creazione del dataset

In [ ]:
def true_function(x):
    return x**3 - x**2 - 5*x 
def create_data(n_samples, noise):
    x = np.linspace(-4, 4, n_samples)
    f = true_function(x)
    y = true_function(x) + noise*np.random.randn(n_samples)
    data = np.zeros((n_samples,3))
    data[:, 0] = x
    data[:, 1] = y
    data[:, 2] = f
    norm = StandardScaler()
    return norm.fit_transform(data)

In [ ]:
np.random.seed(6)
data = create_data(30,10)
X_train = data[:,0][:, np.newaxis]
y_train = data[:,1]
f = data[:,2]

data_test = create_data(20,10)
X_test = data_test[:,0][:, np.newaxis]
y_test = data_test[:,1]

plt.plot(X_train,f, 'k-',alpha=0.6, label='true function')
plt.scatter(X_train,y_train, label='training samples',color='C0',s=50)
plt.scatter(X_test,y_test, label='test samples',color='C1', s=50)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.legend(frameon=True);

# Creazione del modello di predizione

I dati provengono da un modello non lineare, un metodo semplice per creare un modello in grado di fare predizioni non lineari è quello di utilizzare features polinomiali.

Ma quale è il grado delle features polinomiali che meglio approssima la funzione vera?

In [ ]:
def make_features(train_set, degrees):
    train_dict = {}
    for d in degrees:
        train_dict[d] = PolynomialFeatures(d).fit_transform(train_set)
    return train_dict

In [ ]:
degrees=range(21)
train_dict = make_features(X_train, degrees)

In [ ]:
error_train=np.empty(len(degrees))
for d in degrees:
    X_train_poly = train_dict[d]   
    model = LinearRegression()
    model.fit(X_train_poly, y_train)
    y_hat = model.predict(X_train_poly)
    error_train[d] = mean_squared_error(y_train, y_hat)

In [ ]:
bestd = np.argmin(error_train)
plt.plot(degrees, error_train, marker='o', label='train')
plt.plot(bestd, error_train[bestd], marker='*',markersize=25, color='r', label="min train error at d=%d"%bestd, alpha=0.8)
plt.ylabel('Mean Squared Error')
plt.xlabel('degree')
plt.legend(loc=0,frameon=True)
#plt.yscale("log")
plt.xticks(range(21));

Che indicazioni ci da questo grafico?
- che il modello migliore in termini di errore quandratico medio sul nostro training set è il modello di grado 20

# Predizione sui dati di test

Il nostro obiettivo è quello di testare il modello allenato sui dati di train, e il cui ordine è stato scelto dall'analisi precedente, sul dataset di test. 

Dal momento che il modello performa bene sul set di train su cui è stato allenato, è ragionevole aspettarsi che questo modello performi bene anche sul set di test?

In [ ]:
degree = 20
pipeline = make_pipeline(PolynomialFeatures(degree), LinearRegression())
pipeline.fit(X_train, y_train)

In [ ]:
x = np.linspace(-2,2,100)[:, np.newaxis]
plt.plot(x, pipeline.predict(x), linestyle='--',label='learned model',color='C0')
plt.scatter(X_train, y_train, label='train samples',color='C0',s=50)
plt.scatter(X_test, y_test, label='test samples',color='C1',s=50)
plt.plot(X_train, f, 'k-',alpha=0.6, label='true function')
plt.ylim(-4,3)
plt.xlim(np.min(X_train),np.max(X_train))
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.text(-0.5, -2.3, "training score: $MSE$ = {0:.2f}".format(mean_squared_error(y_train,pipeline.predict(X_train))),
         color='C0')
plt.text(-0.5, -2.8, "test score: $MSE$ = {0:.2f}".format(mean_squared_error(y_test,pipeline.predict(X_test))), color='C1')
plt.legend(frameon=True);

In [ ]:
test_error1 = mean_squared_error(y_test,pipeline.predict(X_test))
test_error1

L'errore di test è 3 ordini di grandezza superiore rispetto all'errore di train, come mai si discostano così tanto?

Cerchiamo di capire, tramite un'analisi grafica di sensitività, come variano errore di train ed errore di test al variare dell'ordine del modello.

In [ ]:
r=(1,30)
def plot_poly(degree=1):
    polynomial_features = PolynomialFeatures(degree=degree,
                                             include_bias=False)
    linear_regression = LinearRegression()
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    pipeline.fit(X_train, y_train)

    x = np.linspace(-1.7,1.7,100)[:, np.newaxis]
    plt.plot(x, pipeline.predict(x), linestyle='--', label="learned model")
    plt.plot(X_train, f, 'k-',alpha=0.6,label="true function")
    plt.scatter(X_train, y_train, label="train samples",color='C0',s=50)
    plt.scatter(X_test, y_test, label="test samples",color='C1',s=50)
    plt.xlabel("$x$")
    plt.ylabel("$y$")
    plt.ylim((-4, 3))
    plt.xlim(np.min(X_train),np.max(X_train))
    plt.legend(loc="upper center",frameon=True)
    plt.text(-0.5, -2.3, "training score: $MSE$ = {0:.2f}".format(mean_squared_error(y_train,pipeline.predict(X_train))),
         color='C0')
    plt.text(-0.5, -2.8, "test score: $MSE$ = {0:.2f}".format(mean_squared_error(y_test,pipeline.predict(X_test))), color='C1')
    plt.show()
    
w=interactive(plot_poly, degree=r)
w

Osservazioni dal grafico interattivo:

- i modelli di ordine basso (ordini 1 e 2) non siano in grado di approssimare in maniera sufficientemente accurata la forma del modello vero da cui sono stati generati i dati, si osserva che l'errore di train e l'errore di test sono simili come valori ed entrambi relativamente alti. Si dice che questi modelli soffrano di **bias elevato** o anche di **underfitting**;

- il modello di ordine 3 è in grado di approssimare in maniera accurata la funzione vera, ciò è ragionevole dal momento che il modello vero è appunto una funzione di grado 3. In questa situazione i valori degli errori di train e di test sono simili ed entrambi bassi;

- mano a mano che l'ordine si alza, si osserva che i modelli allenati hanno delle forme sempre più complesse. Questi modelli, oltre ad imparare la forma della funzione vera, tendono ad imparare anche il rumore contenuto nello specifico training set su cui sono allenati. Si osserva che errore di train ed errore di test appaiono molto differenti, in particolare l'errore di train tende a decrescere con l'aumentare dell'ordine del modello (addirittura quando il grado del modello è lo stesso del numero dei dati di train questi ultimi vengono interpolati e si ottiene perciò errore nullo). Al contrario l'errore di test tende ad aumentare in maniera molto elevata all'aumentare dell'ordine del modello. Si dice che questi modelli soffrano di **varianza elevata** o anche di **overfitting**.

## Calcolo degli errori di test di un modello che soffre di bias elevato e di un modello che soffre di varianza elevata

### High variance

In [ ]:
polynomial_features = PolynomialFeatures(degree=20,include_bias=False)
linear_regression = LinearRegression()
pipeline = Pipeline([("polynomial_features", polynomial_features),
                     ("linear_regression", linear_regression)])
pipeline.fit(X_train, y_train)
test_error2 = mean_squared_error(y_test, pipeline.predict(X_test))
test_error2

### High bias

In [ ]:
polynomial_features = PolynomialFeatures(degree=1,include_bias=False)
linear_regression = LinearRegression()
pipeline = Pipeline([("polynomial_features", polynomial_features),
                     ("linear_regression", linear_regression)])
pipeline.fit(X_train, y_train)
test_error3 = mean_squared_error(y_test, pipeline.predict(X_test))
test_error3

# Valutazione di un modello tramite le curve di apprendimento

Un metodo utile per valutare le performances di un modello in termini di bias/varianza è quello di plottare le curve di apprendimento, esse illustrano il processo di apprendimento al variare del numero dei campioni del training set.
Quando un dataset è piccolo, è facile che un modello complesso lo fitti con precisione rischiando però di fare overfitting. Mano a mano che il dataset cresce è ragionevole aspettarsi che l'errore di train aumenti. Al contrario, con un dataset piccolo è facile che il modello allenato su di esso non generalizzi bene su dati nuovi e di conseguenza l'errore di test sarà elevato. Mano a mano che il dataset cresce ci si aspetta che il modello generalizzi meglio e che quindi l'errore di test diminuisca.
Ci forniscono, inoltre, indicazioni empiriche molto utili sul numero di dati di train che dovremmo utilizzare per allenare il modello fissato un ordine tenendo sotto controllo underfitting e overfitting.

In [ ]:
def plot_learning_curves(model, X, y):
    np.random.seed(0)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    train_errors, val_errors = [], []
    for m in range(1, len(X_train)):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train_predict, y_train[:m]))
        val_errors.append(mean_squared_error(y_val_predict, y_val))
    plt.plot((train_errors), "C0", linewidth=3, label="train")
    plt.plot((val_errors), "C1", linewidth=3, label="test")
    plt.ylim(0,2)
    plt.xlabel('training set size')
    plt.ylabel('Mean Squared Error')
    plt.legend(frameon=True)
    plt.show()

In [ ]:
r_degree = (1,30)
np.random.seed(0)
def learning_curves(degree=1):
    data = create_data(200,5)
    X_train = data[:,0][:, np.newaxis]
    y_train = data[:,1]
    polynomial_features = PolynomialFeatures(degree=degree,include_bias=False)
    linear_regression = LinearRegression()
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                             ("linear_regression", linear_regression)])
    plot_learning_curves(pipeline, X_train, y_train)

w=interactive(learning_curves,degree=r_degree)
w    

# Validazione

Una tecnica utilissima per combattere ovefitting e underfitting è quella utilizzare parte dei dati di train per la validazione del modello, questi dati fanno parte di un nuovo dataset che vieno chiamato **dataset di validazione**.

In [ ]:
np.random.seed(6)
data = create_data(30,10)
X_train = data[:,0][:, np.newaxis]
y_train = data[:,1]
f = data[:,2]

data_test = create_data(20,10)
X_test = data_test[:,0][:, np.newaxis]
y_test = data_test[:,1]

X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.25)

plt.plot(X_train,f, 'k-',alpha=0.6, label='true function')
plt.scatter(X_train,y_train, label='training samples',color='C0',s=50)
plt.scatter(X_test,y_test, label='validation samples',color='C1', s=50)
plt.scatter(X_val,y_val, label='validation samples',color='C2',s=100,marker='s',alpha=0.5)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.legend(frameon=True);

In [ ]:
def make_features(train_set, test_set, degrees):
    train_dict = {}
    test_dict = {}
    for d in degrees:
        traintestdict={}
        train_dict[d] = PolynomialFeatures(d).fit_transform(train_set.reshape(-1,1))
        test_dict[d] = PolynomialFeatures(d).fit_transform(test_set.reshape(-1,1))
    return train_dict, test_dict

In [ ]:
degrees=range(21)
train_dict, val_dict = make_features(X_train2, X_val, degrees)

In [ ]:
error_train=np.empty(len(degrees))
error_val=np.empty(len(degrees))

In [ ]:
for d in degrees:
    X_train_poly = train_dict[d]
    X_val_poly = val_dict[d]
    model = LinearRegression()
    model.fit(X_train_poly, y_train2)
    y_train_hat = model.predict(X_train_poly)
    y_val_hat = model.predict(X_val_poly)
    error_train[d] = mean_squared_error(y_train2, y_train_hat)
    error_val[d] = mean_squared_error(y_val, y_val_hat)

In [ ]:
bestd = np.argmin(error_val)
print('Best model degree: ' + str(bestd))
plt.plot(degrees, error_train, marker='o', label='train')
plt.plot(degrees, error_val, marker='o', label='validation',c='C2')
plt.plot(bestd, error_val[bestd], marker='*',markersize=25, color='r', label="min val error at d=%d"%bestd, alpha=0.8)
plt.ylabel('Mean Squared Error')
plt.xlabel('degree')
plt.legend(loc='upper left',frameon=True)
plt.yscale("log")
plt.xticks(range(21));

## Predizione con il modello migliore trovato in fase di validazione

Refitting sull'intero training set e e calcolo dell'errore di test.

In [ ]:
polynomial_features = PolynomialFeatures(degree=bestd,include_bias=False)
linear_regression = LinearRegression()
pipeline = Pipeline([("polynomial_features", polynomial_features),
                     ("linear_regression", linear_regression)])
pipeline.fit(X_train, y_train)
test_error4 = mean_squared_error(y_test, pipeline.predict(X_test))

In [ ]:
errors = ['High variance model','High bias model','Validated model']
sns.stripplot(errors,[test_error2, test_error3, test_error4],s=30)
plt.ylabel('Mean Squared Error');
#plt.yscale("log")

In [ ]:
errors = ['High bias model','Validated model']
sns.stripplot(errors,[test_error3, test_error4],s=30)
plt.ylabel('Mean Squared Error');
plt.yscale("log")

# Cross-validazione

La cross-validazione è una tecnica utilizzabile in presenza di una buona numerosità del campione osservato o training set. In particolare la k-fold cross-validation consiste nella suddivisione del dataset totale in k parti di uguale numerosità e, ad ogni passo, la k-esima parte del dataset viene ad essere il validation dataset, mentre la restante parte costituisce il training dataset. Così, per ognuna delle k parti (di solito k = 10) si allena il modello, evitando quindi problemi di overfitting, ma anche di campionamento asimmetrico (e quindi affetto da bias) del training dataset, tipico della suddivisione del dataset in due sole parti (ovvero training e validation dataset). In altre parole, si suddivide il campione osservato in gruppi di egual numerosità, si esclude iterativamente un gruppo alla volta e lo si cerca di predire con i gruppi non esclusi. Ciò al fine di verificare la bontà del modello di predizione utilizzato.

In [ ]:
np.random.seed(6)
data = create_data(30,10)
X_train = data[:,0][:, np.newaxis]
y_train = data[:,1]
f = data[:,2]

In [ ]:
n_folds=10
degrees=range(21)
train_errors = np.zeros((21,n_folds))
valid_errors = np.zeros((21,n_folds))

In [ ]:
fold = 0
for train, valid in KFold(n_folds, shuffle=True).split(range(30)): 
    train_dict, valid_dict = make_features(X_train[train], X_train[valid], degrees)
    for d in degrees:
        est = LinearRegression()
        est.fit(train_dict[d], y_train[train]) # fit
        train_errors[d, fold] = mean_squared_error(y_train[train], est.predict(train_dict[d])) 
        valid_errors[d, fold] = mean_squared_error(y_train[valid], est.predict(valid_dict[d])) 
    fold += 1

In [ ]:
mean_train_errors = train_errors.mean(axis=1)
mean_valid_errors = valid_errors.mean(axis=1)
std_train_errors = train_errors.std(axis=1)
std_valid_errors = valid_errors.std(axis=1)

In [ ]:
np.random.seed(6)
mindeg = np.argmin(mean_valid_errors)
print('Best model degree: ' + str(mindeg))
post_cv_train_dict, test_dict=make_features(X_train, X_test, degrees)
est = LinearRegression()
est.fit(post_cv_train_dict[mindeg], y_train) 
pred = est.predict(test_dict[mindeg])
err = mean_squared_error(pred, y_test)
errtr=mean_squared_error(y_train, est.predict(post_cv_train_dict[mindeg]))
#plt.errorbar(degrees, [r[0] for r in results], yerr=[r[3] for r in results], marker='o', label='CV error', alpha=0.5)
plt.plot(degrees, mean_train_errors, marker='o', label='CV error train', alpha=0.9)
plt.plot(degrees, mean_valid_errors, marker='o', label='CV error val', alpha=0.9,c='C2')


plt.fill_between(degrees, mean_valid_errors-std_valid_errors, mean_valid_errors+std_valid_errors, color='C2', alpha=0.2)
plt.fill_between(degrees, mean_train_errors-std_train_errors, mean_train_errors+std_train_errors, color='C0', alpha=0.2)


plt.plot([mindeg], [err], 'o',  label='test set error',color='red')#,markersize=25)
plt.xticks(np.arange(21))
plt.ylabel('Mean Squared Error')
plt.xlabel('degree')
plt.legend(loc=0,frameon=True)
plt.yscale("log")

In [ ]:
test_error5=err
errors = ['High variance model','High bias model','Validated model','Cross-validated model']
sns.stripplot(errors,[test_error2, test_error3, test_error4, test_error5],s=30)
plt.ylabel('Mean Squared Error')

In [ ]:
errors = ['High bias model','Validated model','Cross-validated model']
sns.stripplot(errors,[test_error3, test_error4, test_error5],s=30)
plt.ylabel('Mean Squared Error')
plt.yscale('log')

Chiaramente le performances dei modelli ottenuti in fase di validazione e in fase di cross-validazione sono identiche dal momento che il grado ottimale individuato in entrambi i casi era lo stesso.

# Regolarizzazione

In [ ]:
np.random.seed(0)
data = create_data(30,10)
X_train = data[:,0][:, np.newaxis]
y_train = data[:,1]
f = data[:,2]

data_test = create_data(20,10)
X_test = data_test[:,0][:, np.newaxis]
y_test = data_test[:,1]

In [ ]:
r_alpha = (-10,2)
def regularization(alpha=-10):
    alpha=10**alpha
    degrees=range(21)
    fig, col = plt.subplots(1, 2)
    d=19
    train_dict, test_dict = make_features(X_train, X_test, degrees)
    X_train_poly = train_dict[d]
    X_test_poly = test_dict[d]
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_poly, y_train)
    col[0].plot(X_train, f, 'k-',alpha=0.6, label='true function')
    col[0].plot(X_train, y_train, '.', label="training")
    col[0].plot(X_test, y_test, '.', label="testing",color='C1')
    x=np.arange(-1.6,1.6,0.01)
    X = PolynomialFeatures(d).fit_transform(x.reshape(-1,1))
    col[0].plot(x, ridge.predict(X),  '--', label="$\lambda$ = %s" % str(alpha),color='C0')
    #ax.set_ylim((0, 1))
    #ax.set_xlim((0, 1))
    col[0].text(-1.7, 2.3, "training score: $mse$ = {0:.2f}".format(mean_squared_error(y_train,ridge.predict(X_train_poly))),
         color='C0')
    col[0].text(-1.7, 2.6, "test score: $mse$ = {0:.2f}".format(mean_squared_error(y_test,ridge.predict(X_test_poly))), 
                color='C1')
    col[0].set_ylabel('y')
    col[0].set_xlabel('x')
    col[0].legend(loc='lower right')
    col[0].set_ylim(-3,3)
    coef = ridge.coef_.ravel()
    col[1].semilogy(np.abs(coef), marker='o', label="$\lambda$ = %s" % str(alpha))
    col[1].set_ylim((0, 1e4))
    col[1].set_ylabel('abs(coefficient)')
    col[1].set_xlabel('coefficients')
    col[1].legend(loc='upper left')
    col[1].set_xticks=(np.arange(20))
    plt.tight_layout()
    plt.show()
w=interactive(regularization, alpha=r_alpha)
w

## Scelta del parametro $\lambda$ tramite cross-validazione

In [ ]:
np.random.seed(0)
data = create_data(100,10) 
X_train = data[:,0][:, np.newaxis]
y_train = data[:,1]
f = data[:,2]

data_test = create_data(20,10)
X_test = data_test[:,0][:, np.newaxis]
y_test = data_test[:,1]

In [ ]:
np.random.seed(0)
degree = 30
pipeline = Pipeline([('polynomial_features',PolynomialFeatures(degree)),('ridge', Ridge())])
param_grid = [{'ridge__alpha': np.logspace(-8,2,100)}]
grid = GridSearchCV(pipeline, cv=2, n_jobs=-1, param_grid=param_grid,scoring="mean_squared_error")
grid.fit(X_train,y_train)
grid.best_params_

In [ ]:
grid.cv_results_

## Refitting sull'intero dataset

In [ ]:
lambda_best = grid.best_params_['ridge__alpha']
est = Ridge(alpha=lambda_best).fit(X_train,y_train)

In [ ]:
alpha = lambda_best
degrees = range(21)
fig, col = plt.subplots(1, 2)
d=19
train_dict, test_dict = make_features(X_train, X_test, degrees)
X_train_poly = train_dict[d]
X_test_poly = test_dict[d]
ridge = Ridge(alpha=alpha)
ridge.fit(X_train_poly, y_train)
col[0].plot(X_train, f, 'k-',alpha=0.6, label='true function')
col[0].plot(X_train, y_train, '.', label="training")
col[0].plot(X_test, y_test, '.', label="testing",color='C1')
x=np.arange(-1.6,1.6,0.01)
X = PolynomialFeatures(d).fit_transform(x.reshape(-1,1))
col[0].plot(x, ridge.predict(X),'--', label="$\lambda$ = %s" % str(alpha),color='C0')
#ax.set_ylim((0, 1))
#ax.set_xlim((0, 1))
col[0].text(-1.7, 2.3, "training score: $mse$ = {0:.2f}".format(mean_squared_error(y_train,ridge.predict(X_train_poly))),
     color='C0')
col[0].text(-1.7, 2.6, "test score: $mse$ = {0:.2f}".format(mean_squared_error(y_test,ridge.predict(X_test_poly))), 
            color='C1')
col[0].set_ylabel('y')
col[0].set_xlabel('x')
col[0].legend(loc='lower right')
col[0].set_ylim(-3,3)
coef = ridge.coef_.ravel()
col[1].semilogy(np.abs(coef), marker='o', label="$\lambda$ = %s" % str(alpha))
col[1].set_ylim((0, 1e4))
col[1].set_ylabel('abs(coefficient)')
col[1].set_xlabel('coefficients')
col[1].legend(loc='upper left')
col[1].set_xticks=(np.arange(20))
plt.tight_layout()
plt.show()

## Errore di test prima e dopo la regolarizzazione

In [ ]:
polynomial_features = PolynomialFeatures(degree=30,include_bias=False)
regression = LinearRegression()
pipeline = Pipeline([("polynomial_features", polynomial_features),
                     ("linear_regression", regression)])
pipeline.fit(X_train, y_train)
test_error6 = mean_squared_error(y_test, pipeline.predict(X_test))
test_error6

In [ ]:
polynomial_features = PolynomialFeatures(degree=30,include_bias=False)
ridge = Ridge(alpha=lambda_best)
pipeline = Pipeline([("polynomial_features", polynomial_features),
                     ("ridge_regression", ridge)])
pipeline.fit(X_train, y_train)
test_error7 = mean_squared_error(y_test, pipeline.predict(X_test))
test_error7